In [117]:
# import necessary libraries
import pandas as pd
import questionary
from pathlib import Path
import fire 
import numpy as np
import hvplot.pandas
%matplotlib inline

In [118]:
# open csv files using read_csv function
AUDUSD_df = pd.read_csv(
            Path('../Resources/AUDUSD-2000-2020-15m.csv'),
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
EURJPY_df = pd.read_csv(
            Path('../Resources/EURJPY-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
EURUSD_df = pd.read_csv(
            Path('../Resources/EURUSD-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)
USDCHF_df = pd.read_csv(
            Path('../Resources/USDCHF-2000-2020-15m.csv'), 
            index_col = 'DATE_TIME', 
            parse_dates=True, 
            infer_datetime_format=True)

In [119]:
AUDUSD_df.drop(['HIGH', 'LOW', 'OPEN'], axis=1, inplace=True)
EURJPY_df.drop(['HIGH', 'LOW', 'OPEN'], axis=1, inplace=True)
EURUSD_df.drop(['HIGH', 'LOW', 'OPEN'], axis=1, inplace=True)
USDCHF_df.drop(['HIGH', 'LOW', 'OPEN'], axis=1, inplace=True)


In [120]:
#Toggles for the SMA windows. Data is in 15 minutes intervals and 24 hours a day
fast_sma = 10
slow_sma = 20

AUDUSD_df['fast sma'] = AUDUSD_df['CLOSE'].rolling(window=fast_sma).mean()
AUDUSD_df['slow sma'] = AUDUSD_df['CLOSE'].rolling(window=slow_sma).mean()

EURJPY_df['fast sma'] = EURJPY_df['CLOSE'].rolling(window=fast_sma).mean()
EURJPY_df['slow sma'] = EURJPY_df['CLOSE'].rolling(window=slow_sma).mean()

EURUSD_df['fast sma'] = EURUSD_df['CLOSE'].rolling(window=fast_sma).mean()
EURUSD_df['slow sma'] = EURUSD_df['CLOSE'].rolling(window=slow_sma).mean()

USDCHF_df['fast sma'] = USDCHF_df['CLOSE'].rolling(window=fast_sma).mean()
USDCHF_df['slow sma'] = USDCHF_df['CLOSE'].rolling(window=slow_sma).mean()

In [121]:
#adds percent change and cumulative return column for each currency pair
AUDUSD_df['pct_change'] = AUDUSD_df['CLOSE'].pct_change()
AUDUSD_df.dropna(inplace=True)
AUDUSD_df['cum returns'] = (1 + AUDUSD_df['pct_change']).cumprod()

EURJPY_df['pct_change'] = EURJPY_df['CLOSE'].pct_change()
EURJPY_df.dropna(inplace=True)
EURJPY_df['cum returns'] = (1 + EURJPY_df['pct_change']).cumprod()

EURUSD_df['pct_change'] = EURUSD_df['CLOSE'].pct_change()
EURUSD_df.dropna(inplace=True)
EURUSD_df['cum returns'] = (1 + EURUSD_df['pct_change']).cumprod()

USDCHF_df['pct_change'] = USDCHF_df['CLOSE'].pct_change()
USDCHF_df.dropna(inplace=True)
USDCHF_df['cum returns'] = (1 + USDCHF_df['pct_change']).cumprod()

In [122]:
#Adds long/short column. When 'fast sma' is above 'slow sma' strategy is long and vice versa. A 1 indicates the strategy is long and a 0 is short.
#Adds 'strat return' column. If startegy is long, strat return will equal the pct_change column.
#If strategy is short, strat return will be the pct_change multiplied by -1.

AUDUSD_df['long/short'] = np.where(AUDUSD_df['fast sma'] > AUDUSD_df['slow sma'],1,0)
AUDUSD_df['strat return'] = np.where(AUDUSD_df['long/short'] == 1,AUDUSD_df['pct_change'],AUDUSD_df['pct_change'] * -1)
AUDUSD_df.dropna(inplace=True)

#calculates cumulative returns for the strategy based off of the 'strat return' column

AUDUSD_df['cum strat returns'] = (1 + AUDUSD_df['strat return']).cumprod()

#repeat steps above for remaining currency pairs

EURJPY_df['long/short'] = np.where(EURJPY_df['fast sma'] > EURJPY_df['slow sma'],1,0)
EURJPY_df['strat return'] = np.where(EURJPY_df['long/short'] == 1,EURJPY_df['pct_change'],EURJPY_df['pct_change'] * -1)
EURJPY_df.dropna(inplace=True)
EURJPY_df['cum strat returns'] = (1 + EURJPY_df['strat return']).cumprod()

EURUSD_df['long/short'] = np.where(EURUSD_df['fast sma'] > EURUSD_df['slow sma'],1,0)
EURUSD_df['strat return'] = np.where(EURUSD_df['long/short'] == 1,EURUSD_df['pct_change'],EURUSD_df['pct_change'] * -1)
EURUSD_df.dropna(inplace=True)
EURUSD_df['cum strat returns'] = (1 + EURUSD_df['strat return']).cumprod()

USDCHF_df['long/short'] = np.where(USDCHF_df['fast sma'] > USDCHF_df['slow sma'],1,0)
USDCHF_df['strat return'] = np.where(USDCHF_df['long/short'] == 1,USDCHF_df['pct_change'],USDCHF_df['pct_change'] * -1)
USDCHF_df.dropna(inplace=True)
USDCHF_df['cum strat returns'] = (1 + USDCHF_df['strat return']).cumprod()

In [ ]:
#plot cumulative returns
combined_cumulative_returns = pd.concat([
    AUDUSD_df['cum strat returns'],
    EURJPY_df['cum strat returns'],
    EURUSD_df['cum strat returns'],
    USDCHF_df['cum strat returns'],
    ], axis=1, keys=['AUDUSD cumulative', 'EURJPY cumulative', 'EURUSD cumulative', 'USDCHF cumulative'])
combined_cumulative_returns.dropna(inplace=True)
combined_cumulative_returns.hvplot(title=f"Cumulative {fast_sma} and {slow_sma} SMA strategy returns",
                                  xlabel="Year").opts(
                                  yformatter='%.0f')

In [ ]:
#Calculate calendar year returns for the strategy by taking the last and first price of the year in the cumulative returns columns.

start2012 = combined_cumulative_returns.loc['2012-01-02 00:00:00']
end2012 = combined_cumulative_returns.loc['2012-12-31 18:45:00']
annual2012 = (end2012 / start2012) - 1

start2013 = combined_cumulative_returns.loc['2013-01-01 23:00:00']
end2013 = combined_cumulative_returns.loc['2013-12-31 18:45:00']
annual2013 = (end2013 / start2013) - 1

start2014 = combined_cumulative_returns.loc['2014-01-01 23:00:00']
end2014 = combined_cumulative_returns.loc['2014-12-31 19:45:00']
annual2014 = (end2014 / start2014) - 1

start2015 = combined_cumulative_returns.loc['2015-01-19 00:00:00']
end2015 = combined_cumulative_returns.loc['2015-12-31 20:00:00']
annual2015 = (end2015 / start2015) - 1

start2016 = combined_cumulative_returns.loc['2016-01-04 00:00:00']
end2016 = combined_cumulative_returns.loc['2016-12-30 18:45:00']
annual2016 = (end2016 / start2016) - 1

start2017 = combined_cumulative_returns.loc['2017-01-02 09:00:00']
end2017 = combined_cumulative_returns.loc['2017-12-29 20:45:00']
annual2017 = (end2017 / start2017) - 1

start2018 = combined_cumulative_returns.loc['2018-01-02 09:00:00']
end2018 = combined_cumulative_returns.loc['2018-12-31 20:45:00']
annual2018 = (end2018 / start2018) - 1

start2019 = combined_cumulative_returns.loc['2019-01-02 06:00:00']
end2019 = combined_cumulative_returns.loc['2019-12-31 20:45:00']
annual2019 = (end2019 / start2019) - 1

In [ ]:
# Add annual returns to a dataframe

index_labels = ['2012','2013','2014','2015','2016','2017','2018','2019']
column_labels = ['AUDUSD annual return','EURJPY annual return','EURUSD annual return','USDCHF annual return']
annual_returns = pd.DataFrame([annual2012,annual2013,annual2014,annual2015,annual2016,annual2017,annual2018,annual2019],
                        index=index_labels)
annual_returns.columns = column_labels
annual_returns